In [1]:
import os
%pwd
os.chdir("../")

In [2]:
%pwd


'c:\\Users\\rvaib\\OneDrive\\Desktop\\new-D-VS-C'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    batch_size: int
    image_width: int
    image_height: int
    inner_activation: str
    epoch_val: int
    loss_func: str
    optimizer: str
    neurons_no: list


In [4]:
from src.DogvsCatClassifier.constants import *
from src.DogvsCatClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
         self,
         config_filepath = CONFIG_FILE_PATH,
         params_filepath = PARAMS_FILE_PATH):

         self.config =  read_yaml(config_filepath)
         self.params =  read_yaml(params_filepath)   
         create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
         config =self.config.model_trainer
         params =self.params

         create_directories([config.root_dir])

         model_trainer_config = ModelTrainerConfig(
               root_dir=config.root_dir,
               train_data_path=config.train_data_path,
               test_data_path=config.test_data_path,
               model_name=config.model_name,
               batch_size=params.batch_size,
               image_width=params.image_width,
               image_height=params.image_height,
               inner_activation =params.inner_activation,
               epoch_val=params.epoch_val,
               loss_func=params.loss_func,
               optimizer=params.optimizer,
               neurons_no=params.neurons_no

          )

         return model_trainer_config

In [6]:
import joblib
import tensorflow as tf
from tensorflow import keras
from keras import models,layers
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization


In [7]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def set_train_data(self):
        train_ds = keras.utils.image_dataset_from_directory(
            directory = self.config.train_data_path,
            labels='inferred',
            label_mode='int',
            batch_size=self.config.batch_size,
            image_size=(self.config.image_width,self.config.image_height),
            
            
        )
        

        def process(image,label):
            image = tf.cast(image/255. , tf.float32)
            return image,label
        
        train_ds = train_ds.map(process)
        

        return train_ds
        
        
    def model_design(self):
        cnnmodel = models.Sequential([
            layers.Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation=self.config.inner_activation,input_shape=(self.config.image_width,self.config.image_height,3)),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),
            
            layers.Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation=self.config.inner_activation),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),
            
            layers.Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation=self.config.inner_activation),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'),
        
            layers.Flatten(),
            layers.Dense(self.config.neurons_no[0],activation=self.config.inner_activation),
            layers.Dropout(0.1),
            layers.Dense(self.config.neurons_no[1],activation=self.config.inner_activation),
            layers.Dropout(0.1),
            layers.Dense(self.config.neurons_no[7],activation='sigmoid')

            ])
        
        cnnmodel.summary()
        return cnnmodel


    def model_compile(self,train_ds,model):
        model.compile(
            optimizer=self.config.optimizer,
            loss=self.config.loss_func,
            metrics=['accuracy']


        )
        model.fit(train_ds,epochs=self.config.epoch_val)

        
        models.save_model(model, os.path.join(self.config.root_dir,self.config.model_name))
        
        


    

    
    
    

    

        



In [8]:
try:
    config =ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    train_ds=model_trainer.set_train_data()
    model = model_trainer.model_design()
    model_trainer.model_compile(train_ds,model)
    
except Exception as e:
    raise e

[2024-01-31 23:49:07,844: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-31 23:49:07,849: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-31 23:49:07,852: INFO: common: created directory at: artifacts]
[2024-01-31 23:49:07,853: INFO: common: created directory at: artifacts\model_trainer]
Found 20000 files belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
         